In [ ]:
from __future__ import print_function
from __future__ import absolute_import
from keras.models import Sequential
from keras import backend as K
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dense, Dropout, Reshape, Permute, merge, Flatten, concatenate
from keras.callbacks import TensorBoard, ModelCheckpoint
# import keras.layers
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.engine.topology import get_source_inputs
from keras.utils.data_utils import get_file
from keras.utils.layer_utils import convert_all_kernels_in_model
# from imagenet_utils import decode_predictions
#from keras.applications.vgg16 import preprocess_input, decode_prediction
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import VGG16
#from keras.layers import concatenate as concat

# from imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
#from keras.applications.imagenet_utils import _obtain_input_shape
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import img_to_array
from keras.optimizers import Adam
# from __future__ import print_function
# from __future__ import absolute_import
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from imutils import paths
from keras import regularizers

import cv2
import keras.layers
import numpy as np
import keras
import tensorflow as tf


import os, sys
# from imutils import paths
import random
import time
import argparse
import align.detect_face
from keras.preprocessing import image

In [ ]:
def DeXpression(include_top=True,
                weights=None,
                input_tensor=None,
                input_shape=None,
                padding=None,
                classes=6):
    # Check weights
    if weights not in {'dexpression', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `dexpression` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 6:
        raise ValueError('If using `weights` as dexpression with `include_top`'
                         ' as true, `classes` should be 6')
    # Determine proper input shape
    input_shape = _obtain_input_shape(
        input_shape,
        default_size=224,
        min_size=139,
        data_format=K.image_data_format(),
        require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    # START MODEL
    conv_1 = Convolution2D(64, (7, 7), strides=(2, 2), padding=padding, activation='relu', name='conv_1')(img_input)
    maxpool_1 = MaxPooling2D((3, 3), strides=(2,2))(conv_1)
    x = BatchNormalization()(maxpool_1)
    
    # FEAT-EX1
    conv_2a = Convolution2D(96, (1, 1), strides=(1,1), activation='relu', padding=padding, name='conv_2a')(x)
    conv_2b = Convolution2D(208, (3, 3), strides=(1,1), activation='relu', padding=padding, name='conv_2b')(conv_2a)
    maxpool_2a = MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_2a')(x)
    conv_2c = Convolution2D(64, (1, 1), strides=(1,1), name='conv_2c')(maxpool_2a)
    concat_1 = concatenate([conv_2b,conv_2c],axis=3,name='concat_2')
    maxpool_2b = MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_2b')(concat_1)
    
    # FEAT-EX2
    conv_3a = Convolution2D(96, (1, 1), strides=(1,1), activation='relu', padding=padding, name='conv_3a')(maxpool_2b)
    conv_3b = Convolution2D(208, (3, 3), strides=(1,1), activation='relu', padding=padding, name='conv_3b')(conv_3a)
    maxpool_3a = MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_3a')(maxpool_2b)
    conv_3c = Convolution2D(64, (1, 1), strides=(1,1), name='conv_3c')(maxpool_2a)
    concat_3 = concatenate([conv_3b,conv_3c],axis=3,name='concat_3')
    maxpool_3b = MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_3b')(concat_3)
    
    # FINAL LAYERS
    net = Flatten()(maxpool_3b)
    net = Dense(classes, activation='softmax', name='predictions')(net)
    
    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, net, name='deXpression')
    return model

model= DeXpression(include_top=True,
                weights=None,
                input_tensor=None,
                input_shape=None,
                padding='same',
                classes=6)

model.load_weights('/media/topica/DATA/dexpress_kflold/best_result_26_10.h5')

In [ ]:
from keras.models import model_from_json
model_deep = model_from_json(open("/home/topica/workspace/Facial-Expression-Recognition/model_4layer_2_2_pool.json", "r").read())
model_deep.load_weights('/home/topica/workspace/Facial-Expression-Recognition/model_4layer_2_2_pool.h5') #load weights

In [ ]:
emotions = ('angry', 'fear', 'happy', 'sad', 'surprise', 'neutral')

In [ ]:
### ###When you want to dectect multi face
image_size=160
margin= 44
gpu_memory_fraction=1.0
detect_multiple_faces = True

In [ ]:
### ###When you want to dectect multi face
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
frame = 0
cap = cv2.VideoCapture(0)
count = np.zeros((6))

In [ ]:
### ###When you want to dectect multi face
while(True):
    ret, img = cap.read()
    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
    factor = 0.709 # scale factor
    start_time = time.time()
    bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
    nrof_faces = bounding_boxes.shape[0]
    if nrof_faces>0:
        have_face = True
        det = bounding_boxes[:,0:4]
        det_arr = []
        result_face = []
        img_size = np.asarray(img.shape)[0:2]
        if nrof_faces>1:
            if detect_multiple_faces:
                for i in range(nrof_faces):
                    det_arr.append(np.squeeze(det[i]))
            else:
                bounding_box_size = (det[:,2]-det[:,0])*(det[:,3]-det[:,1])
                img_center = img_size / 2
                offsets = np.vstack([ (det[:,0]+det[:,2])/2-img_center[1], (det[:,1]+det[:,3])/2-img_center[0] ])
                offset_dist_squared = np.sum(np.power(offsets,2.0),0)
                index = np.argmax(bounding_box_size-offset_dist_squared*2.0) # some extra weight on the centering
                det_arr.append(det[index,:])
        else:
            det_arr.append(np.squeeze(det))

        for i, det in enumerate(det_arr):
            det = np.squeeze(det)
            bb = np.zeros(4, dtype=np.int32)
            bb[0] = np.maximum(det[0]-margin/2, 0)
            bb[1] = np.maximum(det[1]-margin/2, 0)
            bb[2] = np.minimum(det[2]+margin/2, img_size[1])
            bb[3] = np.minimum(det[3]+margin/2, img_size[0])
            cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
#             scaled = misc.imresize(cropped, (image_size, image_size), interp='bilinear')
            cv2.rectangle(img,(bb[0],bb[1]),(bb[2],bb[3]),(164,64,64),2) #highlight detected face
            #detected_face = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY) #transform to gray scale
            detected_face = cv2.resize(cropped, (224, 224)) #resize to 48x48
#             print(detected_face.shape)
            img_pixels = image.img_to_array(detected_face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)

            img_pixels /= 255 #pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]

            #-----------------------------

            predictions = model.predict(img_pixels) #store probabilities of 7 expressions
            max_index = np.argmax(predictions[0])
            ###deepcnn
            detected_face_deep = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY) #transform to gray scale
            detected_face_deep = cv2.resize(detected_face_deep, (48, 48)) #resize to 48x48
            img_pixels_deep = image.img_to_array(detected_face_deep)
            img_pixels_deep = np.expand_dims(img_pixels_deep, axis = 0)

            img_pixels_deep /= 255 #pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]
            predictions_deep = model_deep.predict(img_pixels_deep) #store probabilities of 7 expressions
            max_index_deep = np.argmax(predictions_deep[0])
            
            
            count[max_index] = count[max_index]+1
            if (max_index_deep == 4):
                result = "%s %s%s" % (emotions_deep[max_index_deep], round(predictions_deep[0][max_index_deep]*100, 2), '%')
            elif (predictions[0][max_index] >= 0.75):
                result = "%s %s%s" % (emotions[max_index], round(predictions[0][max_index]*100, 2), '%')
            else :
                result = "neutral"
            
            color = (55,55,255)
            cv2.putText(img, result, (int(bb[0]+5), int(bb[1]-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1) 
        print("--- %s seconds ---" % (time.time() - start_time))
    cv2.imshow('img',img)
    frame = frame + 1
    if cv2.waitKey(70) & 0xFF == ord('q'): #press q to quit
        break

#kill open cv things
# out.release()
cap.release()
cv2.destroyAllWindows()
print("the general emotion of the class: " + emotions[np.argsort(count)[4]])

In [ ]:
count